# Special operations on indexes
https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes

下面一些额外的操作，不是所有类型的index都支持的.

# Reconstructing vectors from an index

reconstruct 和 reconstruct_n 方法通过 `ids` 从 `index` 中重建一个或多个 `vector(s)`.

示例: [test_index_composite.py](https://github.com/facebookresearch/faiss/blob/master/tests/test_index_composite.py#L35)

支持 IndexFlat, IndexIVFFlat (call make_direct_map first), IndexIVFPQ (same), IndexPreTransform (provided the underlying transform supports it)

In [6]:
import numpy as np
import faiss

index_file_name='test.index'

d = 64                           
nb = 100000                       
np.random.seed(1234)                              # 对生成的数据，每次都一样，可预见，若无此句，每次都不一样
xb = np.random.random((nb, d)).astype('float32')  # 向量集合，矩阵nb*d
xb[:, 0] += np.arange(nb) / 1000                  # 修正向量集合第一列，每项加 0...nb/1000 小数
ids = np.arange(100000, 100000+nb).astype('int')

index = faiss.index_factory(d, "IVF100,Flat", faiss.METRIC_INNER_PRODUCT)
index.nprobe = 20
index.train(xb)
# init make_direct_map，and reconstruct
index.make_direct_map()
index_id = faiss.IndexIDMap2(index)
index_id.add_with_ids(xb, ids)
# save
faiss.write_index(index_id, index_file_name)

In [9]:
# load
new_index = faiss.read_index(index_file_name)
uid=100000
k=10
try:
    re_vector=new_index.reconstruct(uid)
    print(re_vector)
    D, I = new_index.search(re_vector.reshape(1,-1), k) 
    print(I)
except RuntimeError:
    print('{} not exist.'.format(uid))

[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926
 0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951
 0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845
 0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824
 0.9331401  0.65137815 0.39720258 0.78873014 0.31683612 0.56809866
 0.8691274  0.4361734  0.8021476  0.14376682 0.70426095 0.7045813
 0.21879211 0.92486763 0.44214076 0.90931594 0.05980922 0.18428709
 0.04735528 0.6748809  0.59462476 0.5333102  0.04332406 0.5614331
 0.32966843 0.5029668  0.11189432 0.6071937  0.5659447  0.00676406
 0.6174417  0.9121229  0.7905241  0.99208146 0.95880175 0.7919641
 0.28525096 0.62491673 0.4780938  0.19567518]
[[119039 118150 114750 120525 116302 111500 118313 118297 115098 116258]]


# Removing elements from an index

The method remove_ids removes a subset of vectors from an index. It takes an IDSelector object that is called for every element in the index to decide whether it should be removed. IDSelectorBatch will do this for a list of indices. The Python interface constructs this efficiently.

NB that since it does a pass over the whole database, this is efficient only when a significant number of vectors needs to be removed.

示例: [test_index_composite.py](https://github.com/facebookresearch/faiss/blob/master/tests/test_index_composite.py#L25)

支持 IndexFlat, IndexIVFFlat, IndexIVFPQ, IDMap.

Note that there is a semantic difference when removing ids from sequential indexes vs. when removing them from an IndexIVF:

- for sequential indexes (IndexFlat, IndexPQ, IndexLSH), the removal operation shifts the ids of vectors above the removed vector id.
- the IndexIVFs store the ids of vectors explicitly, so the ids of other vectors are not changed.

# Range search

The method range_search returns all vectors within a radius around the query point (as opposed to the k nearest ones). Since the result lists for each query are of different sizes, it must be handled specially:

- in C++ it returns the results in a pre-allocated RangeSearchResult[https://github.com/facebookresearch/faiss/blob/master/AuxIndexStructures.h#L35] structure
- in Python, the results are returned as a triplet of 1D arrays lims, D, I, where result for query i is in I[lims[i]:lims[i+1]] (indices of neighbors), D[lims[i]:lims[i+1]] (distances).

Supported by (CPU only): IndexFlat, IndexIVFFlat, IndexScalarQuantizer, IndexIVFScalarQuantizer.

# Splitting and merging indexes

The methods:

merge_from copies another index to this and deallocates it on-the-fly. You can use ivflib::merge_into for IndexIVFs wrapped in a pre-transform.

- copy_subset_to copies a subset of this codes to another index. Example usage: [to build indexes on a GPU and move them to CPU afterwards](https://github.com/facebookresearch/faiss/blob/master/benchs/bench_gpu_1bn.py#L541)
- The functions are implemented only for IndexIVF subclasses because they are mainly interesting for large indexes.